In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [10]:
df = pd.read_csv('/kaggle/input/credit-score-data/preprocessed_train.csv')
df_test = pd.read_csv('/kaggle/input/credit-score-data/preprocessed_test.csv')

In [11]:
df.head()

,Month,Age,Profession,Income_Annual,Base_Salary_PerMonth,Total_Bank_Accounts,Total_Credit_Cards,Rate_Of_Interest,Total_Current_Loans,Delay_from_due_date,...,Current_Debt_Outstanding,Ratio_Credit_Utilization,Credit_History_Age,Payment_of_Min_Amount,Per_Month_EMI,Monthly_Investment,Payment_Behaviour,Monthly_Balance,Credit_Score,Loan_Count
0,7,51,11,101583.48,8648.290000,5,7,10,4,8,...,50.93,34.462154,289,0,190.811017,630.015789,3,314.002193,1,4
1,1,23,14,101926.95,8635.912500,4,4,9,1,13,...,1058.00,39.693812,245,0,70.587681,662.803927,4,410.199642,1,1
2,2,49,14,158871.12,12962.260000,0,4,8,1,8,...,576.48,39.367225,228,0,86.905860,746.805985,4,742.514154,1,1
3,6,40,3,60379.28,4804.606667,5,6,18,3,15,...,725.39,29.061701,205,0,90.906385,166.418658,1,473.135623,1,3
4,5,17,0,50050.83,4085.902500,9,10,20,5,28,...,3419.10,30.386321,54,1,190.445060,56.789441,0,401.355749,0,5


In [12]:
X = df.drop('Credit_Score', axis=1)
y = df['Credit_Score']
X_test = df_test.drop('ID', axis=1)

In [13]:
device = 'cuda'

In [14]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
# Convert data to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).long()  
X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val.to_numpy()).long()  

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.3):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Define hyperparameters
input_size = X.shape[1]  # Number of features
hidden_size = 256  # Hidden layer size
num_classes = 3  # Number of output classes
dropout_rate = 0.4  # Dropout rate
num_epochs = 20  # Number of epochs
learning_rate = 0.001  # Learning rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, criterion, optimizer, and scheduler
model = NeuralNetwork(input_size, hidden_size, num_classes, dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    scheduler.step()  # Update learning rate
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
y_pred, y_true = [], []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Epoch [1/20], Loss: 0.7590
Epoch [2/20], Loss: 0.7316
Epoch [3/20], Loss: 0.7231
Epoch [4/20], Loss: 0.7214
Epoch [5/20], Loss: 0.7172
Epoch [6/20], Loss: 0.7075
Epoch [7/20], Loss: 0.7033
Epoch [8/20], Loss: 0.7026
Epoch [9/20], Loss: 0.7033
Epoch [10/20], Loss: 0.7009
Epoch [11/20], Loss: 0.6993
Epoch [12/20], Loss: 0.6991
Epoch [13/20], Loss: 0.7001
Epoch [14/20], Loss: 0.6990
Epoch [15/20], Loss: 0.6992
Epoch [16/20], Loss: 0.6994
Epoch [17/20], Loss: 0.6995
Epoch [18/20], Loss: 0.6984
Epoch [19/20], Loss: 0.6985
Epoch [20/20], Loss: 0.6986
Validation Accuracy: 68.10%


In [15]:
X_test_scaled = scaler.transform(X_test)

In [16]:
# Convert the scaled test data to a PyTorch tensor
X_test_tensor = torch.from_numpy(X_test_scaled).float().to(device)

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(X_test_tensor)  # Get the raw outputs from the model
    _, y_test_pred = torch.max(outputs, 1)  # Get the predicted class with highest score

# Convert the predictions back to a NumPy array
y_test = y_test_pred.cpu().numpy()

print(y_test)

[0 0 0 ... 1 2 0]


In [17]:
y_pred_test = pd.Series(y_test).map({0: 'Poor', 1: 'Standard', 2: 'Good'})
neural_model = pd.DataFrame({'ID': df_test['ID'], 'Credit_Score': y_pred_test})
neural_model.head()

,ID,Credit_Score
0,0x2145,Poor
1,0x7d59,Poor
2,0xe753,Poor
3,0x19813,Standard
4,0x1dc5e,Standard


In [18]:
neural_model.to_csv('neural_submission.csv', index=False)